### As you all may know by now, CV and LB are not very well correlated for this competition, in order to investigate this further, i tried analyzing where my model predictions went wrong.


In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from ast import literal_eval


In [ ]:
BASE_PATH = '../input/shopee-product-matching/train_images'

device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
oof = pd.read_csv("../input/shopee-oofs/eff_b0_oof.csv")

In [ ]:
oof['preds'] = oof.preds.apply(lambda x: literal_eval(str(x)))
oof['preds2'] = oof.preds2.apply(lambda x: literal_eval(str(x)))
oof['preds3'] = oof.preds3.apply(lambda x: literal_eval(str(x)))


oof['oof'] = oof['oof'].apply(lambda x: ", ".join(x.split(" ")))
oof['target'] = oof['target'].apply(lambda x: ", ".join(x.split(" ")))

oof['oof'] = oof.oof.apply(lambda x: literal_eval(str(x)))
oof['target'] = oof.target.apply(lambda x: literal_eval(str(x)))

In [ ]:
def display_images(imgs): 
    f, ax = plt.subplots(1,len(imgs), figsize=(20,5))
    
    for index, (i, row) in enumerate(imgs.iterrows()) :                
        image = cv2.imread(f"{BASE_PATH}/{row.image}")
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
        ax[index].imshow(image) 
        ax[index].axis('off') 
                
        brand = row.posting_id
        
        
        title_len = 30
        if len(imgs) > 5:
            title_len = 10
        
        ax[index].set_title(f"{brand}\n{row.title[:title_len]}", fontsize="12")
    plt.show() 

In [ ]:
def fetch_results(row):
    
    if len(row.target) < 10 and (len(row.oof) > 1 and len(row.oof) < 10):
        print("Query Image (posting_id): ", row.posting_id)
        image = cv2.imread(f"{BASE_PATH}/{row.image}")
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)        
        plt.imshow(image) 

        targets = oof.loc[oof.posting_id.isin(row.target)]
        preds = oof.loc[oof.posting_id.isin(row.oof)]

        print("target images: ", len(targets))
        display_images(targets)

        print("predicted images: ", len(preds))
        display_images(preds)

## Taking rows where f1 score is less than 0.6

In [ ]:
miss_df = oof.loc[oof.f1 < 0.6]

## Few Examples where target labels are not fully correct and label predicted from model make much more sense 

In [ ]:
fetch_results(miss_df.iloc[0])

In [ ]:
fetch_results(miss_df.iloc[1])

In [ ]:
fetch_results(miss_df.iloc[7])

In [ ]:
fetch_results(miss_df.iloc[11])

In [ ]:
fetch_results(miss_df.iloc[19])

In [ ]:
fetch_results(miss_df.iloc[27])

In [ ]:
fetch_results(miss_df.iloc[36])

In [ ]:
fetch_results(miss_df.iloc[43])

In [ ]:
fetch_results(miss_df.iloc[67])

In [ ]:
fetch_results(miss_df.iloc[69])

In [ ]:
fetch_results(miss_df.iloc[71])

## Above shown examples are just few for demonstration, actual number of noisy labels is much bigger.